In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive


Mounted at /content/gdrive
'Colab Notebooks'      'Internship-Aug2020-Gowtham R.pdf'   Photos
'Getting started.pdf'  'My Drive'			   'Pongal 2021'


In [2]:
%cd /content/gdrive/MyDrive/Colab Notebooks/Stock_Value_Prediction_Final_Year_Project

/content/gdrive/MyDrive/Colab Notebooks/Stock_Value_Prediction_Final_Year_Project


In [3]:
from keras.models import load_model
import pandas as pd
import numpy as np 
from os.path import join
from myFunctions.candle_stick_plot import plotCandleStick
from myFunctions.list_data_dir import load_data_files
from myFunctions import dataConversion 
from myFunctions.predictionPlot import prediction_plot
from sklearn.model_selection import train_test_split
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasRegressor,KerasClassifier
from sklearn.model_selection import RandomizedSearchCV,KFold

In [4]:
fields = ['Date','High','Low','Open','Close']
lookback = 30
datafolder = r"/content/gdrive/MyDrive/Colab Notebooks/Stock_Value_Prediction_Final_Year_Project/Data/NIFTY50"
prediction_date = pd.Timestamp(2020,11,30,00,00,00)

In [5]:
df = pd.read_csv(r"/content/gdrive/MyDrive/Colab Notebooks/Stock_Value_Prediction_Final_Year_Project/Data/NIFTY50/ADANIPORTS.csv",usecols = fields)
df['Date'] = pd.to_datetime(df['Date'])
df.head()

,Date,Open,High,Low,Close
0,2007-11-27,770.00,1050.00,770.0,962.90
1,2007-11-28,984.00,990.00,874.0,893.90
2,2007-11-29,909.00,914.75,841.0,884.20
3,2007-11-30,890.00,958.00,890.0,921.55
4,2007-12-03,939.75,995.00,922.0,969.30


In [6]:
Open_X , Open_y = dataConversion.convert2matrix(df['Open'],lookback)

In [7]:
Open_X_train, Open_X_test , Open_y_train , Open_y_test = train_test_split(Open_X,Open_y,test_size=0.2)

In [8]:
def create_model(layers=1 ,neurons=256):
    model=Sequential()

    model.add(Dense(500,activation='relu',input_shape=(lookback,)))

    for i in range(layers):
        model.add(Dense(neurons,activation='relu'))
    model.add(Dense(1,activation='relu'))
    model.compile(optimizer='adam',loss='mean_squared_error',metrics=['mse'])
    return model

In [9]:
model = KerasRegressor(build_fn=create_model,epochs=5,batch_size=16)
params = dict(layers=[1,2,3,4,5],neurons=[5,25,50,100,150,200,250,300])
random_search = RandomizedSearchCV(model,params,cv=KFold(3))
random_search.fit(Open_X_train,Open_y_train)

Epoch 1/5
107/107 [==============================] - 1s 4ms/step - loss: 143273.0062 - mse: 143273.0062
Epoch 2/5
107/107 [==============================] - 0s 3ms/step - loss: 137333.5515 - mse: 137333.5515
Epoch 3/5
107/107 [==============================] - 0s 3ms/step - loss: 154850.8087 - mse: 154850.8087
Epoch 4/5
107/107 [==============================] - 0s 3ms/step - loss: 138020.6306 - mse: 138020.6306
Epoch 5/5
54/54 [==============================] - 0s 1ms/step - loss: 140980.0312 - mse: 140980.0312
Epoch 1/5
107/107 [==============================] - 1s 3ms/step - loss: 12925.4709 - mse: 12925.4709
Epoch 2/5
107/107 [==============================] - 0s 4ms/step - loss: 1606.9220 - mse: 1606.9220
Epoch 3/5
107/107 [==============================] - 0s 3ms/step - loss: 1525.9486 - mse: 1525.9486
Epoch 4/5
107/107 [==============================] - 0s 3ms/step - loss: 1409.3330 - mse: 1409.3330
Epoch 5/5
54/54 [==============================] - 0s 1ms/step - loss: 286.8618 

RandomizedSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=False),
                   error_score=nan,
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7f006bc61490>,
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'layers': [1, 2, 3, 4, 5],
                                        'neurons': [5, 25, 50, 100, 150, 200,
                                                    250, 300]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=0)

In [10]:
random_search.best_params_

{'layers': 2, 'neurons': 200}

In [11]:
random_search.best_score_

-640.9390258789062

In [12]:
best_open_model = random_search.best_estimator_

In [13]:
best_open_model.model.evaluate(Open_X_test,Open_y_test)

20/20 [==============================] - 0s 1ms/step - loss: 274.1202 - mse: 274.1202


[274.12017822265625, 274.12017822265625]

In [14]:
best_open_model.model.save("/content/gdrive/MyDrive/Colab Notebooks/Stock_Value_Prediction_Final_Year_Project/models/open_model")

INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/Colab Notebooks/Stock_Value_Prediction_Final_Year_Project/models/open_model/assets
